# 畳み込み符号

In [1]:
import numpy as np
from scipy.sparse.csgraph import shortest_path
import itertools

In [4]:
class MyCircuit:
    """
    回路
    """

    def __init__(self, status_num=0):
        # 畳み込み 問2
        # シフトレジスタ
        self.sr = [[0], [0, 0]]
        # 生成系列
        self.g = [
            [[1, 1], [0, 0, 0]],
            [[0, 1], [0, 1, 1]],
            [[1, 0], [1, 0, 1]]
        ]
        
        # 授業資料 例1
#         self.sr = [[0, 0, 0]]
#         self.g = [
#             [[1, 0, 1, 1]],
#             [[1, 1, 1, 1]]
#         ]
        
        # 授業資料 例3
        # 畳み込み 問1と同じ
        self.sr = [[0, 0]]
        self.g = [
            [[1, 1, 0]],
            [[1, 0, 1]],
            [[1, 1, 1]]
        ]
        
        # メモリ数
        self.v = sum(len(i) for i in self.sr)
        # 最大のシフトレジスタ長
        self.m = max(len(i) for i in self.sr)
        # 同時入力数??
        self.u_len = len(self.sr)
        # 同時出力数
        self.w_len = len(self.g)
        self._set_sr(status_num)
        self.status_num = self._calc_status_num()
    
    def transition(self, u_list):
        """
        状態遷移
        現在の状態で、入力を与えると出力を返す
        次の状態へ遷移する
        """
        w_list = []
        for g_list in self.g:
            w = 0
            for i, u in enumerate(u_list):
                sr_vec = np.array([u] + self.sr[i])
                g_vec = np.array(g_list[i])
                w += np.dot(sr_vec, g_vec)
            w_list.append(w % 2)
        for i in range(len(self.sr)):
            self.sr[i] = [u_list[i]] + self.sr[i][:-1]
        self.status_num = self._calc_status_num()
        return w_list
    
    def _calc_status_num(self):
        """
        レジスタの中身から状態番号計算
        """
        return int("".join([str(i) for i in itertools.chain.from_iterable(self.sr)]), 2)
    
    def _set_sr(self, status_num):
        """
        シフトレジスタの中身を指定した状態に対応させる
        """
        idx = 0
        st_bin_str = format(status_num, f"0{self.v}b")
        for i in range(len(self.sr)):
            for j in range(len(self.sr[i])):
                self.sr[i][j] = int(st_bin_str[idx])
                idx += 1
    
    def __str__(self):
        moji = f"S_{self.status_num}\n"
        for i, j in enumerate(self.sr):
            moji += f"sr{i}: {j}\n"
        return moji[:-1]

def create_state_transition_dict():
    """
    状態遷移の辞書を作成
    """
    c = MyCircuit(1)
    # 全状態
    st_tr_dic = {i : {} for i in range(1 << c.v)}
    # 全入力パターン
    act_list = [[int(j) for j in format(i, f"0{c.u_len}b")] for i in range(1 << c.u_len)]
    for st in st_tr_dic.keys():
        for u_list in act_list:
            c = MyCircuit(st)
            w_list = c.transition(u_list)
            st_tr_dic[st][c.status_num] = [u_list, w_list]
    return st_tr_dic

def to_adjacency_matrix(st_tr_dic: dict):
    """
    状態遷移の辞書から隣接行列作成
    重み付き有効グラフとする
    重みは出力のハミング重み
    """
    m = np.zeros([len(st_tr_dic)] * 2)
    for src, child_dic in st_tr_dic.items():
        for dst, uw in child_dic.items():
            weight = sum(uw[1])
            # 重み0は辺が無いと判定されるため微小な値を入れる
            if weight == 0:
                weight = 1e-5
            m[src][dst] = weight
    return m

def get_path(src, dst, p):
    sp = []
    p_row = p[src]
    i = dst
    while i != src and i >= 0:
        sp.append(i)
        i = p_row[i]
    if i < 0:
        sp = []
    else:
        sp.append(i)
    return sp[::-1]

In [5]:
incidence_dict = create_state_transition_dict()
for k, v in incidence_dict.items():
    print(k, v)
m = to_adjacency_matrix(incidence_dict)
# print(m)
saitan_w, saitan_p = shortest_path(m, method="D", return_predecessors=True)
saitan_w = saitan_w.astype("u1")
print(saitan_w)
print(saitan_p)
# print(get_path(2, 6, saitan_p))

0 {0: [[0], [0, 0, 0]], 2: [[1], [1, 1, 1]]}
1 {0: [[0], [0, 1, 1]], 2: [[1], [1, 0, 0]]}
2 {1: [[0], [1, 0, 1]], 3: [[1], [0, 1, 0]]}
3 {1: [[0], [1, 1, 0]], 3: [[1], [0, 0, 1]]}
[[0 5 3 4]
 [2 0 1 2]
 [4 2 0 1]
 [4 2 3 0]]
[[-9999     2     0     2]
 [    1 -9999     1     2]
 [    1     2 -9999     2]
 [    1     3     1 -9999]]


In [6]:
# 最短自由距離を計算
# S_0からS_iの間を行って帰ってくるまでの最短距離を計算
# i>0について繰り返し、最小のものが最短自由距離。多分
d_free = float("inf")
d_free_path = []

for i in range(1, saitan_w.shape[0]):
    cycle_len_min_i = saitan_w[0][i] + saitan_w[i][0]
    cycle_path_min_i = get_path(0, i, saitan_p) + get_path(i, 0, saitan_p)[1:]
    print(f"i={i}, length_min={cycle_len_min_i}")
    print(f"path={cycle_path_min_i}")
    if cycle_len_min_i < d_free:
        d_free = cycle_len_min_i
        d_free_path = cycle_path_min_i
print(f"最小自由距離: {d_free}, 経路: {d_free_path}")

i=1, length_min=7
path=[0, 2, 1, 0]
i=2, length_min=7
path=[0, 2, 1, 0]
i=3, length_min=8
path=[0, 2, 3, 1, 0]
最小自由距離: 7, 経路: [0, 2, 1, 0]


In [7]:
# 符号化
u = "11101"
# u = "10111"
# u = "1001011"
# u = "11011001"
# u = "01011001"
u_list = [int(i) for i in u]
code_str = ""
code_list = []

c = MyCircuit(0)
while c.status_num != 0 or u_list:
    if u_list:
        u_block = u_list[:c.u_len]
    else:
        u_block = [0] * c.u_len
    # 状態遷移
    w_block = c.transition(u_block)
    print(u_block)
    print(c)
    print(w_block)
    print("#" * 30)
    code_str += "".join([str(i) for i in w_block]) + " "
    code_list += w_block
    u_list = u_list[c.u_len:]
# print(code_list)
print(f"符号系列: {code_str}")

[1]
S_2
sr0: [1, 0]
[1, 1, 1]
##############################
[1]
S_3
sr0: [1, 1]
[0, 1, 0]
##############################
[1]
S_3
sr0: [1, 1]
[0, 0, 1]
##############################
[0]
S_1
sr0: [0, 1]
[1, 1, 0]
##############################
[1]
S_2
sr0: [1, 0]
[1, 0, 0]
##############################
[0]
S_1
sr0: [0, 1]
[1, 0, 1]
##############################
[0]
S_0
sr0: [0, 0]
[0, 1, 1]
##############################
符号系列: 111 010 001 110 100 101 011 


# ビダビアルゴリズム

In [8]:
r = "110110110111010101101"

# 畳み込み問1.6
r = "101111011110101000100110011"
c = MyCircuit()

# 更新用辞書
min_path_dict_now = {0 : [0, []]}

r_list = [int(i) for i in r]
# r_list = code_list
while r_list:
    r_block = np.array(r_list[:c.w_len])
    min_path_dict_next = {}
    for st_now, v in min_path_dict_now.items():
        for st_next, uw in incidence_dict[st_now].items():
            e_weight = (r_block ^ np.array(uw[1])).sum()
#             print(st_now, st_next, uw[1], r_block, e_weight)
            total_e_weight = v[0] + e_weight
            if st_next in min_path_dict_next:
                if total_e_weight < min_path_dict_next[st_next][0]:
                    min_path_dict_next[st_next][0] = total_e_weight
                    min_path_dict_next[st_next][1] = v[1] + [st_now]
            else:
                min_path_dict_next[st_next] = [total_e_weight]
                min_path_dict_next[st_next].append(v[1] + [st_now])
    min_path_dict_now = min_path_dict_next
    print(min_path_dict_now)
    r_list = r_list[c.w_len:]

min_path = min_path_dict_now[0][1] + [0]
est_series = []
est_str = ""
dec_series = []

st_now = 0
for i in range(1, len(min_path)):
    st_next = min_path[i]
    u_list, w_list = incidence_dict[st_now][st_next]
    est_series += w_list
    est_str += "".join([str(i) for i in w_list]) + " "
    dec_series += u_list
    st_now = st_next

# メモリ数間引く
dec_str = "".join([str(i) for i in dec_series[:-(c.m * c.u_len)]])

print(f"推定系列: {est_str}")
print(f"情報系列: {dec_str}")

{0: [2, [0]], 2: [1, [0]]}
{0: [5, [0, 0]], 2: [2, [0, 0]], 1: [2, [0, 2]], 3: [3, [0, 2]]}
{0: [2, [0, 2, 1]], 2: [5, [0, 2, 1]], 1: [4, [0, 0, 2]], 3: [3, [0, 0, 2]]}
{0: [4, [0, 2, 1, 0]], 2: [3, [0, 2, 1, 0]], 1: [3, [0, 0, 2, 3]], 3: [6, [0, 2, 1, 2]]}
{0: [5, [0, 0, 2, 3, 1]], 2: [4, [0, 0, 2, 3, 1]], 1: [3, [0, 2, 1, 0, 2]], 3: [6, [0, 2, 1, 0, 2]]}
{0: [5, [0, 0, 2, 3, 1, 0]], 2: [4, [0, 2, 1, 0, 2, 1]], 1: [6, [0, 0, 2, 3, 1, 2]], 3: [5, [0, 0, 2, 3, 1, 2]]}
{0: [6, [0, 0, 2, 3, 1, 0, 0]], 2: [6, [0, 0, 2, 3, 1, 2, 1]], 1: [5, [0, 2, 1, 0, 2, 1, 2]], 3: [6, [0, 2, 1, 0, 2, 1, 2]]}
{0: [7, [0, 2, 1, 0, 2, 1, 2, 1]], 2: [6, [0, 2, 1, 0, 2, 1, 2, 1]], 1: [6, [0, 2, 1, 0, 2, 1, 2, 3]], 3: [7, [0, 0, 2, 3, 1, 2, 1, 2]]}
{0: [6, [0, 2, 1, 0, 2, 1, 2, 3, 1]], 2: [8, [0, 2, 1, 0, 2, 1, 2, 1, 0]], 1: [8, [0, 2, 1, 0, 2, 1, 2, 1, 2]], 3: [7, [0, 2, 1, 0, 2, 1, 2, 1, 2]]}
推定系列: 111 101 011 111 101 100 010 110 011 
情報系列: 1001011


In [32]:
c.m

2